<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from numba import njit, float64, int64, boolean
from numba.types import unicode_type
from numba.experimental import jitclass
from utils import gen_GBM
from talib import BBANDS, SMA

N = 100000
price = gen_GBM(N)
upper, middle, lower = BBANDS(price, 20, 1, 1)
sma_slow = SMA(price, 20)
sma_fast = SMA(price, 6)

price = price[20:]
upper = upper[20:]
middle = middle[20:]
lower = lower[20:]
sma_slow = sma_slow[20:]
sma_fast = sma_fast[20:]

In [2]:
@njit(fastmath=True)
def crossup(x,y,i):
    if x[i - 1] < y[i - 1] and x[i] > y[i]:
        return 1
    else:
        return 0

    
@njit 
def crossdown(x,y,i):
    if x[i - 1] > y[i - 1] and x[i] < y[i]:
        return 1
    else:
        return 0
    
@njit 
def gt(x,y,i):
    if x[i] > y[i]:
        return 1
    else:
        return 0
    
@njit 
def lt(x,y,i):
    if x[i] < y[i]:
        return 1
    else:
        return 0

spec = [
    ('x',float64[:]),
    ('y',float64[:]),
    ('how',unicode_type),
    ('acc', boolean),
    ('_acc', int64),
    ('i', int64)
]

@jitclass(spec)
class Signal:
    
    def __init__(self, x, y, how, acc):
        self.x = x
        self.y = y
        if how is None or how not in ["crossup", "crossdown", "lower", "higher"]:
            raise ValueError(
                'how must be one of "crossup", "crossdown", "lower", or "higher"'
            )
        else:
            self.how = how
        self.acc = acc
        self._acc = 0
    
    def sig(self, i):
        
        if self.how == "crossup":
            out = crossup(self.x, self.y, i)
                
        elif self.how == "crossdown":
            out = crossdown(self.x, self.y, i)
            
        elif self.how == "higher":
            out = gt(self.x, self.y, i)

        elif self.how == "lower":
            out = lt(self.x, self.y, i)
            
        if self.acc:
            self._acc += out
            return self._acc
        else:
            return out

In [3]:
sig1 = Signal(price, lower, "crossup", True)

out = sig1.sig(10)
%timeit sig1.sig(10)

# 662 ns ± 2.25 ns  with jitclass
# 551 ns ± 1.55 ns wo jitclass

659 ns ± 3.18 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [4]:
%timeit sig1.sig(10) & sig1.sig(111)

1.35 µs ± 3.14 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%timeit getattr(operator, 'and_')(sig1.sig(10),sig1.sig(111))

1.41 µs ± 0.74 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [9]:
import operator
getattr(operator, 'and_')(True, False)

False

In [4]:
x = crossup(price, lower, 10)
%timeit crossup(price, lower, 10)

# 303 ns ± 0.924

306 ns ± 0.103 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [5]:
# @njit
def gen_ret(p, l):    
    sig = Signal(price, lower, "crossup", True)
    L = p.shape[0]
    out = np.empty(L, dtype=np.int64)
    out[0] = 0
    for i in range(1,L):
        out[i] = sig.sig(1)
    return out

out_ = gen_ret(price, lower)
%timeit gen_ret(price, lower)

# 2.36 ms ± 763 ns

72.7 ms ± 517 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
@njit(fastmath=True, boundscheck = 0)
def gen_ret(p, l):    
    sig = Signal(price, lower, "crossup", True)
    L = p.shape[0]
    out = np.empty(L, dtype=np.int64)
    out[0] = 0
    for i in range(1,L):
        out[i] = sig.sig(1)
    return out

out_ = gen_ret(price, lower)
%timeit gen_ret(price, lower)

# 2.36 ms ± 946 ns

2.39 ms ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
x = np.random.randn(100000)

In [8]:
%timeit x.sum()

33.2 µs ± 7.62 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [9]:
@njit(fastmath=True)
def func(x):
    s = 0.0
    L = x.shape[0]
    for i in range(L):
        s += x[i]
    return s
        
%timeit func(x)

18 µs ± 22.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [12]:
from numba import typeof
typeof(True), typeof('str')

(bool, unicode_type)